In [1]:
import numpy as np
import os
from sklearn.metrics import confusion_matrix
import seaborn as sn; sn.set(font_scale=1.4)
from sklearn.utils import shuffle           
import matplotlib.pyplot as plt             
import cv2                                 
import tensorflow as tf
import random
from tqdm import tqdm
from PIL import Image
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.optimizers import SGD, Adam

In [2]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device



In [ ]:
class_names = ['good','bad']
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}

nb_classes = len(class_names)

IMAGE_SIZE = (224, 224)

def load_data():
    datasets = ['train', 'test']#資料夾
    output = []
    
    # Iterate through training and test sets
    for dataset in datasets:
        
        images = []
        labels = []
        
        print("Loading {}".format(dataset))
        
        # Iterate through each folder corresponding to a category
        for folder in os.listdir(dataset):
            label = class_names_label[folder]
            
            # Iterate through each image in our folder
            for file in tqdm(os.listdir(os.path.join(dataset, folder))):
                
                # Get the path name of the image
                img_path = os.path.join(os.path.join(dataset, folder), file)
                
                # Open and resize the img
                image = cv2.imread(img_path)
                #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                #cv讀照片，顏色莫認為BGR，需轉為RGB，錯誤表示黑白或已轉
                image = cv2.resize(image, IMAGE_SIZE)  
                #image = Image.convert("L")
                # Append the image and its corresponding label to the output
                images.append(image)
                labels.append(label)
                
        images = np.array(images, dtype = 'float32')
        labels = np.array(labels, dtype = 'int32')   
        
        output.append((images, labels))

    return output

(train_images, train_labels), (test_images, test_labels) = load_data()

In [ ]:
train_images.shape

In [7]:
#隨機性
train_images, train_labels = shuffle(train_images, train_labels, random_state=25)
#標準化
train_images = train_images / 255.0 
test_images = test_images / 255.0
#建模
input_shape = (224, 224, 3)

model = Sequential([
    Conv2D(64, (3, 3), input_shape=input_shape, padding='same',activation='relu', strides=2),
    Conv2D(64, (3, 3), input_shape=input_shape, padding='same',activation='relu', strides=2),
    MaxPooling2D(pool_size=(2, 2), strides=2),
    Dropout(0.5),
    
    Conv2D(128, (3, 3), input_shape=input_shape, padding='same',activation='relu', strides=2),
    Conv2D(128, (3, 3), input_shape=input_shape, padding='same',activation='relu', strides=2),
    MaxPooling2D(pool_size=(2, 2), strides=2),
    Dropout(0.5),
    
    Conv2D(256, (3, 3), input_shape=input_shape, padding='same',activation='relu', strides=2),
    MaxPooling2D(pool_size=(2, 2), strides=2),
    Dropout(0.5),
    
    Flatten(),#平坦層
    
    Dense(units=512, activation='relu'),#隱藏層1
    Dropout(0.5),
    
    Dense(units=512, activation='relu'),#隱藏層2
    Dropout(0.5),
    
    Dense(2, activation='softmax') #輸出層，分類用softmax，只分好壞是2
])
model.compile(optimizer = 'adam', #SGD(lr=0.1)
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(x=train_images,y=train_labels,epochs=30,batch_size=200,verbose=1)

In [ ]:
#模型概況
plt.title('train_loss')
plt.ylabel('loss')
plt.xlabel('Epoch')
plt.plot(history.history["loss"])
#scores = model.evaluate(test_images,test_labels)
#print('test:',result[1])
#預測
predictions = model.predict(test_images)#Vector of probabilities
pred_labels = np.argmax(predictions, axis = 1)

In [13]:
CM = confusion_matrix(test_labels,pred_labels)
def accuracy(confusion_matrix):
    diagonal_sum = confusion_matrix.trace()
    sum_of_all_elements =confusion_matrix.sum()
    return diagonal_sum / sum_of_all_elements
print(accuracy(CM))

0.8486111111111111


In [ ]:
CM

In [ ]:
#混淆矩陣視覺化,看錯誤
ax = plt.axes()
sn.heatmap(CM, annot=True,
            annot_kws={"size": 10},
            xticklabels=class_names,
            yticklabels=class_names, ax = ax)
ax.set_title ('Confusion matrix')
plt.show()

In [15]:
#存模型&讀模型
model.save("ColonModel.h5")

In [16]:
from tensorflow.keras.models import load_model
model = load_model('ColonModel.h5')

In [ ]:
IMAGE_path=r'C:\Users\dl\Desktop\yc_data\colon_AI\test\good\good11_35.jpg'
img=tf.keras.preprocessing.image.load_img(IMAGE_path,target_size=(224,224))
img=tf.keras.preprocessing.image.img_to_array(img)
#img=tf.image.rgb_to_grayscale(img) 
plt.imshow(img/255.)

In [30]:
prediction=model.predict(np.array([img]),verbose=0)
print(prediction)
if prediction[0,0]>prediction[0,1]:
    print("good")
else:
    print("bad")

    

[[0. 1.]]
bad


In [17]:
prediction[0,1]

0.0011339865